In [26]:
import os
from pathlib import Path
import sys

import IPython.display as ipd
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
#import seaborn as sns
import sklearn as skl
import sklearn.utils, sklearn.preprocessing, sklearn.decomposition, sklearn.svm
import librosa
import librosa.display
import ast
import warnings

dir = os.getcwd()
# go one level higher and add it to path
new_dir = Path(dir).parents[0]
sys.path.append(str(new_dir))

from src.config import AUDIO_DIR
from src.data.data_utils import load
from src.features.features import get_features

pd.set_option('display.max_columns', 52)

In [27]:
try:
    features = pd.read_csv('../data/interim/features.csv', index_col=0, header=[0, 1, 2])
except:
    print('save features file in folder /data/interim !')
    # to create features:
    #get_features('small')

In [21]:
features = features.dropna()
features.shape

(7951, 518)

In [28]:
try:
    targets = pd.read_csv('../data/fma_metadata/tracks.csv', index_col=0, header=[0, 1, 2]).loc[features.index, ('track', 'genre_top')]
except:
    print('save tracks file in folder /data/fma_metadata !')

In [23]:
features

feature    chroma_cens                                                     \
statistics    kurtosis                                                      
number              01        02        03        04        05         06   
track_id                                                                    
2            -0.217359 -0.726509 -0.491446 -0.721771 -0.545604   0.903521   
5            -0.575293 -0.460787 -1.310206 -0.371414 -0.279014  -0.456130   
10           -0.170648 -0.662701  0.334679 -0.269012 -0.633766  -0.800806   
140          -0.286780  1.582875  0.664635  0.370429 -0.525093  -1.094748   
141           0.337887 -0.320074 -0.225522 -0.831719 -0.411882  -1.212733   
...                ...       ...       ...       ...       ...        ...   
154308       -1.053601  0.060788  1.704757 -0.777857  0.558388  -0.662202   
154309        2.169788 -1.155132 -0.303650 -1.045133 -1.208173  -0.825725   
154413       -1.219351 -1.417390  0.208791  4.801033 -0.561867   3.271480   
154414       -0.239980 -0.642887  1.149441 -0.793276 -1.293259  -1.022434   
155066        1.009953 -0.091882 -1.310494 -1.732816 -1.094380  20.023039   

feature                                                                 \
statistics                                                               
number            07        08        09        10        11        12   
track_id                                                                 
2           0.786983  0.917156 -0.793144  0.452647 -0.229356 -0.173608   
5          -0.788048  0.126617 -0.816839  3.057143  0.065180  2.229395   
10         -0.514046 -0.019774 -0.477296 -0.710341 -0.347353 -0.336416   
140         0.839174 -0.202234 -0.432132 -1.217938 -0.682920 -0.605614   
141        -0.553260 -0.743021  0.849655  0.155225 -0.050075  0.660437   
...              ...       ...       ...       ...       ...       ...   
154308     -1.358890 -0.714282 -1.047748  2.689562 -1.196743  2.795735   
154309     -0.481393  6.507251  0.381729  1.691741  2.469630  1.070482   
154413      5.800009  9.027393 -1.526886  1.665599 -0.886765  3.377588   
154414     -1.177673  1.347859 -0.231648 -0.943169 -0.604957  1.652884   
155066      9.381350  0.520161 -0.901654  0.246841  0.411799  5.859467   

feature                                                                 \
statistics       max                                                     
number            01        02        03        04        05        06   
track_id                                                                 
2           0.731929  0.564340  0.509567  0.539360  0.799817  0.504063   
5           0.503388  0.507648  0.693599  0.569417  0.537767  0.622475   
10          0.413630  0.472618  0.667295  0.654727  0.442657  0.415954   
140         0.521293  0.862354  0.590986  0.683515  0.608712  0.654876   
141         0.519412  0.429041  0.616609  0.877509  0.498666  0.873961   
...              ...       ...       ...       ...       ...       ...   
154308      0.458248  0.612681  0.972323  1.000000  0.859181  0.566026   
154309      0.824763  0.821103  0.886571  0.476689  0.514607  0.658231   
154413      0.896096  0.882674  0.870927  0.940653  0.940773  0.888128   
154414      0.621285  0.680503  0.691272  0.731936  0.770049  0.721503   
155066      0.269397  0.969931  0.970135  0.861873  0.571166  0.728939   

feature                                                                 \
statistics                                                               
number            07        08        09        10        11        12   
track_id                                                                 
2           0.563302  0.427014  0.529112  0.422337  0.501536  0.506892   
5           0.526745  0.844188  0.546589  0.513516  0.325426  0.529823   
10          0.509747  0.336658  0.499810  0.456472  0.474244  0.352999   
140         0.740992  0.684725  0.793263  0.802711  0.814004  0.769616   
141         0.729198  0.654637

In [24]:
targets

track_id,Unnamed: 40_level_2
track_id,
2,Hip-Hop
5,Hip-Hop
10,Pop
140,Folk
141,Folk
...,...
154308,Hip-Hop
154309,Hip-Hop
154413,Pop
